In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

/var/folders/mc/sd8lk0kn4fz1q41w5qdn2fhc0000gn/T/ipykernel_83771/2683037679.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## Introduction

Here we will compute the basic formulae for options in __Black and Scholes model.__

One of the assumptions of the model is that the considered stock follows a Geometric Brownian motion. It incorporates two key components: the drift ($\mu$) and the volatility ($\sigma$). The drift represents the expected return of the stock, while the volatility measures the standard deviation of the returns. This model assumes continuous evolution of the stock price over time.

Stocks dynamics are described by the following equation: 

$$dS_t = \mu S_t dt  + \sigma S_t dW_t$$

- $dS_t$ represents the change in the stock price at time $t$.
- $\mu S_t$ captures the expected growth of the stock.
- $\sigma S_t dW_t$represents the random component due to the Brownian motion (Wiener process)
- $dt$ denotes a small change in time.
- $dW$ is a random variable with mean zero and variance $dt$.


Under this model, and using the fact that there is no arbitrage opportunity, the value at $t$ of an european call option with strike $K$, maturity $T$ and constant risk-free rate $r$ is given by the formula:

$$V_t = S_t N(d_1) - Ke^{-r(T-t)}N(d_2)$$

The function $N(x)$ represents the cumulative distribution function for the standarized normal distribution. 

where
$$d_1 = \frac{log(\frac{S}{K}) + (r + \frac{1}{2}\sigma^2)(T-t)}{\sigma \sqrt{T - t}}$$

and
$$d_1 = \frac{log(\frac{S}{K}) + (r - \frac{1}{2}\sigma^2)(T-t)}{\sigma \sqrt{T - t}}$$

*For the put value we will use the call-put parity.*

In [2]:
def norm_cdf(x):
    return (1 + math.erf(x / math.sqrt(2))) / 2

def norm_pdf(x):
    return math.exp(-x**2 / 2) / math.sqrt(2 * math.pi)

In [3]:
def euro_values(S0, K, r, sigma, T, D=0):
    # Computing d1 and d2 as defined before
    d1 = (math.log(S0 / K) + (r - D + sigma**2 / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    # Computing european call price
    call_price = S0 * math.exp(-D * T) * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

    # Computing put price using call-put parity
    put_price = call_price - S0 * math.exp(-D * T) + K * math.exp(-r * T)

    # Call Greeks
    call_delta = math.exp(-D * T) * norm_cdf(d1)
    call_gamma = (math.exp(-D * T) * norm_pdf(d1)) / (S0 * sigma * math.sqrt(T))
    call_theta = (-S0 * math.exp(-D * T) * norm_pdf(d1) * sigma) / (2 * math.sqrt(T)) - r * K * math.exp(-r * T) * norm_cdf(d2) + D * S0 * math.exp(-D * T) * norm_cdf(d1)
    call_vega = S0 * math.exp(-D * T) * norm_pdf(d1) * math.sqrt(T)
    call_rho = K * T * math.exp(-r * T) * norm_cdf(d2)

    # Put Greeks (also computed using call-put parity)
    put_delta = call_delta - math.exp(-D * T)
    put_gamma = call_gamma
    put_theta = call_theta + D * S0 * math.exp(-D * T) - r * K * math.exp(-r * T)
    put_vega = call_vega
    put_rho = -K * T * math.exp(-r * T) * norm_cdf(-d2)

    return call_price, put_price, call_delta, call_gamma, call_theta, call_vega, call_rho, put_delta, put_gamma, put_theta, put_vega, put_rho


In [4]:
def calculate_option_values(stock_price, strike_price, time_to_maturity, option_type):
    stock_prices = np.linspace(0.1, 2 * stock_price, 100)
    option_values_stock = []
    time_to_maturities = np.linspace(0.01, 5, 100)
    option_values_time = []
    option_deltas = []
    option_gammas = []
    option_thetas = []
    for price in stock_prices:
        call_price, put_price, call_delta, call_gamma, call_theta, call_vega, call_rho, put_delta, put_gamma, put_theta, put_vega, put_rho = euro_values(
            price, strike_price, 0.05, 0.2, time_to_maturity)
        if option_type == "call":
            option_values_stock.append(call_price)
            option_deltas.append(call_delta)
            option_gammas.append(call_gamma)
            option_thetas.append(call_theta)
        else:
            option_values_stock.append(put_price)
            option_deltas.append(put_delta)
            option_gammas.append(put_gamma)
            option_thetas.append(put_theta)
    for ttm in time_to_maturities:
        call_price, put_price, _, _, _, _, _, _, _, _, _, _ = euro_values(
            stock_price, strike_price, 0.05, 0.2, ttm)
        if option_type == "call":
            option_values_time.append(call_price)
        else:
            option_values_time.append(put_price)
    return stock_prices, option_values_stock, time_to_maturities, option_values_time, option_deltas, option_gammas, option_thetas

In [ ]:
app = dash.Dash(__name__)

# Definir la función para calcular los valores de la opción en función del stock price

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("European Options Dashboard under Black and Scholes model"),
    html.H2("Parameters"),
    html.Div([
        html.Label("Time to Maturity"),
        dcc.Slider(
            id="time-to-maturity-slider",
            min=0,
            max=5,
            step=0.1,
            value=1,
            marks={i: str(i) for i in range(0, 6)}
        )
    ]),
    html.Div([
        html.Label("Strike Price"),
        dcc.Slider(
            id="strike-price-slider",
            min=0,
            max=100,
            step=1,
            value=50,
            marks={i: str(i) for i in range(0, 101, 10)}
        )
    ]),
    html.Div([
        html.Label("Stock Price"),
        dcc.Input(
            id="stock-price-input",
            type="number",
            value=50
        )
    ]),
    html.Div([
        html.Label("Option Type"),
        dcc.RadioItems(
            id="option-type-radio",
            options=[
                {"label": "Call", "value": "call"},
                {"label": "Put", "value": "put"}
            ],
            value="call"
        )
    ]),
    html.H3("Options Value"),
        html.Div(children=[
        dcc.Graph(id="options-value-graph", style={'display': 'inline-block'}),
        dcc.Graph(id="options-value-vs-time-graph", style={'display': 'inline-block'})
]),
    html.H3("Greeks"),
        html.Div(children=[
        dcc.Graph(id="delta-graph", style={'display': 'inline-block'}),
        dcc.Graph(id="gamma-graph", style={'display': 'inline-block'}),
        dcc.Graph(id="theta-graph", style={'display': 'inline-block'})
])
    ])

# Definir la función de actualización del gráfico de opciones
@app.callback(
    [Output("options-value-graph", "figure"),
     Output("options-value-vs-time-graph", "figure"),
     Output("delta-graph", "figure"),
     Output("gamma-graph", "figure"),
     Output("theta-graph", "figure")],
    [Input("time-to-maturity-slider", "value"),
     Input("strike-price-slider", "value"),
     Input("stock-price-input", "value"),
     Input("option-type-radio", "value")]
)
def update_options_value_graph(time_to_maturity, strike_price, stock_price, option_type):
    
    
    stock_prices, option_values_stock, time_to_maturities, option_values_time, option_deltas, option_gammas, option_thetas = calculate_option_values(
        stock_price, strike_price, time_to_maturity, option_type)
    
    if option_type == "call":
        current_value = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[0]
        current_delta = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[2]
        current_gamma = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[3]
        current_theta = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[4]
    else:
        current_value = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[1]
        current_delta = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[7]
        current_gamma = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[8]
        current_theta = euro_values(stock_price, strike_price, 0.05, 0.2, time_to_maturity)[9]
        
    fig_stock = {
        "data": [
            {
                "x": stock_prices,
                "y": option_values_stock,
                "mode": "lines",
                "name": "Option Value", 
                "line": {"color": "rgb(0, 0, 255)"}
            },
            {
                "x": [stock_price],
                "y": [current_value],
                "mode": "markers",
                "marker": {"size": 10},
                "name": "Current Stock Price"
            }
        ],
        "layout": {
            "title": " Value vs. Stock Price",
            "xaxis": {"title": "Stock Price"},
            "yaxis": {"title": "Option Value"}, 
            "showgrid": False
        }
    }
    
    fig_time = {
        "data": [
            {
                "x": time_to_maturities,
                "y": option_values_time,
                "mode": "lines",
                "name": "Option Value", 
                "line": {"color": "rgb(0, 0, 255)"}
            },
            {
                "x": [time_to_maturity],
                "y": [current_value],
                "mode": "markers",
                "marker": {"size": 10},
                "name": "Current Time to Maturity"
            }
        ],
        "layout": {
            "title": "Value vs. Time to Maturity",
            "xaxis": {"title": "Time to Maturity (in Yrs.)"},
            "yaxis": {"title": "Option Value"},
            "showgrid": False
        }
    }
    
    fig_delta = {
        "data": [
            {
                "x": stock_prices,
                "y": option_deltas,
                "mode": "lines",
                "name": "Delta", 
                "line": {"color": "rgb(0, 0, 255)"}
            },
            {
                "x": [stock_price],
                "y": [current_delta],
                "mode": "markers",
                "marker": {"size": 10},
                "name": "Current Delta"
            }
        ],
        "layout": {
            "title": " Delta vs. Stock Price",
            "xaxis": {"title": "Stock Price"},
            "yaxis": {"title": "Delta"}, 
            "showgrid": False
        }
    }
    fig_gamma = {
        "data": [
            {
                "x": stock_prices,
                "y": option_gammas,
                "mode": "lines",
                "name": "Delta", 
                "line": {"color": "rgb(0, 0, 255)"}
            },
            {
                "x": [stock_price],
                "y": [current_gamma],
                "mode": "markers",
                "marker": {"size": 10},
                "name": "Current Gamma"
            }
        ],
        "layout": {
            "title": "Gamma vs. Stock Price",
            "xaxis": {"title": "Stock Price"},
            "yaxis": {"title": "Gamma"}, 
            "showgrid": False
        }
    }
    fig_theta = {
        "data": [
            {
                "x": stock_prices,
                "y": option_thetas,
                "mode": "lines",
                "name": "Theta", 
                "line": {"color": "rgb(0, 0, 255)"}
            },
            {
                "x": [stock_price],
                "y": [current_theta],
                "mode": "markers",
                "marker": {"size": 10},
                "name": "Current Theta"
            }
        ],
        "layout": {
            "title": "Theta vs. Stock Price",
            "xaxis": {"title": "Stock Price"},
            "yaxis": {"title": "Theta"}, 
            "showgrid": False
        }
    }
    
    return fig_stock, fig_time, fig_delta, fig_gamma, fig_theta

# Ejecutar la aplicación Dash
if __name__ == "__main__":
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_favicon.ico?v=2.10.2 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 15:29:36] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Jun/2023 15:29:39] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
plotting_option_values(option_type = "P")

## The Greeks


In [ ]:
for maturity in np.linspace(0.01, 2, 10):
    list_prices = []
    for stock_price in np.linspace(30, 70, 100):
        list_prices.append(euro_values(stock_price, 50, 0.05, 0.2, maturity)[2])
    plt.plot(np.linspace(30, 80, 100), list_prices)
plt.title("Delta Change with Time")

In [ ]:
euro_values(1200, 1140, 0.06, 0.30, 1/2,0.03)